In [1]:
# Imports
import pandas as pd
import numpy as np

url1 = "https://datasets.imdbws.com/title.basics.tsv.gz"
url2 = "https://datasets.imdbws.com/title.akas.tsv.gz"
url3 = "https://datasets.imdbws.com/title.ratings.tsv.gz"


# Cleaning Akas Data

In [2]:
# Load akas Data
akas = pd.read_csv(url2, sep='\t', low_memory=False)

In [3]:
akas.replace({'\\N':np.nan}, inplace=True)
akas = akas[(akas['region'] == 'US')]
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1360806 entries, 5 to 33526010
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1360806 non-null  object
 1   ordering         1360806 non-null  int64 
 2   title            1360806 non-null  object
 3   region           1360806 non-null  object
 4   language         3693 non-null     object
 5   types            964828 non-null   object
 6   attributes       45194 non-null    object
 7   isOriginalTitle  1359431 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.4+ MB


# Cleaning title.basics dataset

In [4]:
# Load basics Data
basics = pd.read_csv(url1, sep='\t', low_memory=False)

In [5]:
# Replace '\N' with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [6]:
# Drop any rows that have null values in either genre or runtimeMinutes.
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

In [7]:
# Drop all rows where titleType!=Movie. Keep only titleType==Movie
is_Movie = basics['titleType'] == 'movie'
basics = basics[is_Movie]

In [8]:
# Drop all rows where the start year is not between 2000-2022. Keep startYear 2000-2022

# We first need to change the column type to something numeric.
print("\nData Type before conversion : ", basics['startYear'].dtype)
basics['startYear'] = pd.to_numeric(basics['startYear'])
print("\nData Type after conversion : ", basics['startYear'].dtype)

#Now we can make and apply our filters
filter2000 = basics['startYear'] >= 2000 
filter2022 = basics['startYear'] <=2022
basics = basics[filter2000 & filter2022]


Data Type before conversion :  object

Data Type after conversion :  float64


In [9]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [10]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers_basics = basics['tconst'].isin(akas['titleId'])
basics = basics[keepers_basics]

basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82979 entries, 34792 to 9305865
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82979 non-null  object 
 1   titleType       82979 non-null  object 
 2   primaryTitle    82979 non-null  object 
 3   originalTitle   82979 non-null  object 
 4   isAdult         82979 non-null  object 
 5   startYear       82979 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  82979 non-null  object 
 8   genres          82979 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.3+ MB


# Lets clean ratings

In [11]:
# Load ratings Data
ratings = pd.read_csv(url3, sep='\t', low_memory=False)

In [12]:
ratings.replace({'\\N':np.nan}, inplace=True)
keepers = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]

ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472011 entries, 0 to 1238831
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         472011 non-null  object 
 1   averageRating  472011 non-null  float64
 2   numVotes       472011 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.4+ MB


# save clean files

In [13]:
# Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)